In [44]:
import numpy as np

MetaID = 0
MetaDATA = 1
MetaLABEL = 2
MetaSTART = 3
MetaDURATION = 4

print("Loading data...")
metadata = np.loadtxt('HT_Sensor_metadata.dat', skiprows=1, dtype=str)
dataset = np.loadtxt('/mnt/ExtDrive/sensor data/HT_Sensor_dataset.dat', skiprows=1)
print("Preprocessing data...")

# data[0] - Linear, no background reduced
# data[1] - Linear, background reduced
# data[2] - Quadratic, no background reduced
# data[3] - Quadratic, background reduced
banana_data = [[], [], [], []]
wine_data = [[], [], [], []]
background_data = [[], [], [], []]

def sqr_features(features, length):
    features_sqr = np.resize(features, length)
    idx = len(features)
    for i in range(0, len(features)):
        for j in range(0, i):
            features_sqr[idx] = features[i] * features[j]
            idx += 1
    return features_sqr

def sqr_features_str(features):
    num_features = len(features)
    for i in range(0, num_features):
        for j in range(0, i):
            features.append(features[i] + "*" + features[j])

feature_names1 = ["time", "R1", "R2", "R3", "R4", "R5", "R6", "R7", "R8", "temp", "h"]
feature_names2 = feature_names1.copy()
for i in range(0, len(feature_names2)):
    feature_names2.append(feature_names2[i] + "'")
sqr_features_str(feature_names1)
sqr_features_str(feature_names2)


segment_len = 1 / 60 # one minute increments
next_segment = 0
cur_ind = None
cur_segment_data = []
cur_background_data = None
p_data = None
for entry in dataset:
    if cur_ind is None or int(entry[0]) != int(cur_ind[MetaID]):
        cur_ind = metadata[int(entry[0])]
        cur_background_data = None
        cur_segment_data = []
        next_segment = 0
        
        if cur_ind[MetaLABEL] == 'banana':
            p_data = banana_data
        elif cur_ind[MetaLABEL] == 'wine':
            p_data = wine_data
        else:
            p_data = background_data
        
    cur_segment_data.append(entry[1:])
    if entry[1] < next_segment or entry[1] > float(cur_ind[MetaDURATION]):
        continue
    
    if cur_background_data is None:
        cur_background_data = np.mean(np.array(cur_segment_data), axis=0)
    else:
        p_data[0].append(entry[1:])
        p_data[1].append(np.concatenate((entry[1:], entry[1:] - cur_background_data)))
        p_data[2].append(sqr_features(p_data[0][len(p_data[0])-1], len(feature_names1)))
        p_data[3].append(sqr_features(p_data[1][len(p_data[1])-1], len(feature_names2)))
    
    cur_segment_data = []
    next_segment += segment_len

print("Done")
#print("PCA 1...")
#all_data = np.concatenate((banana_data1, wine_data1, background_data1))
#all_data_z = all_data - np.mean(all_data, axis=0)
#all_data_z /= np.std(all_data_z, axis=0)
#print(all_data_z.shape)
#U1, s1, VT1 = np.linalg.svd(all_data_z, full_matrices=False)
#print(s1)

#print("PCA 2...")
#all_data = np.concatenate((banana_data2, wine_data2, background_data2))
#all_data_z = all_data - np.mean(all_data, axis=0)
#all_data_z /= np.std(all_data_z, axis=0)
#U2, s2, VT2 = np.linalg.svd(all_data_z, full_matrices=False)
#print(s2)

#   0: id
#   1: time
# 2-9: R1-R8
#  10: temperature
#  11: humidity

Loading data...
Preprocessing data...


In [43]:
p = VT2[0,:]
for i in range(0, len(p)):
    if np.abs(p[i]) > 0.09:
        print(feature_names2[i], p[i])

print(np.array(banana_data1).shape)

R1 -0.2888919899032882
R2 -0.2882943639015939
R3 -0.3024432220608651
R4 -0.29242006319976677
R6 -0.18083318938094894
R7 -0.17204875055541094
R8 -0.16162397671454648
R1' -0.31365514452433246
R2' -0.3105910064569595
R3' -0.32515044168217655
R4' -0.3176143120809689
R6' -0.24145050213878544
R7' -0.19185123355249034
R8' -0.18000230284235677
temp' 0.13596751162853493
(1375, 11)


In [47]:

def ridge_regression(test_data, background_data):
    # Combine positive samples with background samples.
    Xb = np.concatenate((test_data, background_data))
    yb = np.concatenate((np.full(len(test_data), 1), np.full(len(background_data), -1)))

    # Shuffle so that the background samples aren't all in the back.
    rng_state = np.random.get_state()
    np.random.shuffle(Xb)
    np.random.set_state(rng_state)
    np.random.shuffle(yb)

    num_samples, num_features = Xb.shape
    NUM_PARTS = 8

    # Set up partitions
    setIndices = [[0, int(num_samples / NUM_PARTS)]]
    for i in range(1, NUM_PARTS):
        prev = setIndices[i-1][1]
        setIndices.append([prev, int(prev+num_samples/NUM_PARTS)])
    setIndices[NUM_PARTS-1][1] = num_samples

    holdOuts = [[0, 1]]
    for i in range(1, NUM_PARTS - 2):
        prev = holdOuts[i-1][1]
        holdOuts.append([prev, prev+1])

    lam_vals = [0.1, 0.2, 0.4, 1, 2, 4, 10, 20, 40, 100, 200, 400]

    errorRate = 0
    for j in range(0, len(holdOuts)):
        v1Ind = np.arange(setIndices[holdOuts[j][0]][0], setIndices[holdOuts[j][0]][1])
        v2Ind = np.arange(setIndices[holdOuts[j][1]][0], setIndices[holdOuts[j][1]][1])
        trnInd = list(set(range(len(yb))) - set(v1Ind) - set(v2Ind))

        Xt = Xb[trnInd,:]
        yt = yb[trnInd]

        Xv1 = Xb[v1Ind,:]
        yv1 = yb[v1Ind]
        Xv2 = Xb[v2Ind,:]
        yv2 = yb[v2Ind]

        minErr = float("inf")
        minIdx = 0
        minW = []
        for i in range(0, len(lam_vals)):
            w = np.linalg.inv(Xt.T@Xt + lam_vals[i] * np.identity(num_features))@Xt.T@yt

            error = np.abs(np.sign(Xv1@w) - yv1).sum()
            if error <= minErr:
                minErr = error
                minIdx = i
                minW = w
        #print("Min lam:", lam_vals[minIdx])
        #print("Min err:", minErr)
        errorRate += np.abs(np.sign(Xv2@minW) - yv2.flatten()).sum() / (2 * len(yv2))

    errorRate /= len(holdOuts)    
    return errorRate

print("Without background-removed features:")
print("  Linear:")
print("    Banana:", ridge_regression(banana_data[0], background_data[0]))
print("    Wine:", ridge_regression(wine_data[0], background_data[0]))
print("  Quadratic:")
print("    Banana:", ridge_regression(banana_data[2], background_data[2]))
print("    Wine:", ridge_regression(wine_data[2], background_data[2]))
print("With background-removed features:")
print("  Linear:")
print("    Banana:", ridge_regression(banana_data[1], background_data[1]))
print("    Wine:", ridge_regression(wine_data[1], background_data[1]))
print("  Quadratic:")
print("    Banana:", ridge_regression(banana_data[3], background_data[3]))
print("    Wine:", ridge_regression(wine_data[3], background_data[3]))

Without background-removed features:
  Linear:
    Banana: 0.1448349307774228
    Wine: 0.09618794326241135
  Quadratic:
    Banana: 0.4547390841320554
    Wine: 0.3687943262411348
With background-removed features:
  Linear:
    Banana: 0.06389776357827474
    Wine: 0.07668439716312057
  Quadratic:
    Banana: 0.44728434504792336
    Wine: 0.36436170212765956
